In [1]:
import os
import numpy as np
import xarray as xr
from xhistogram.xarray import histogram
from cmocean import cm

from matplotlib import pyplot as plt
%matplotlib inline

import mitequinox.utils as ut
import mitequinox.plot as pl

In [2]:
from dask.distributed import Client, LocalCluster
#
#cluster = LocalCluster()
#
from dask_jobqueue import PBSCluster
cluster = PBSCluster() #processes=7, cores=7
w = cluster.scale(jobs=17)
#
client = Client(cluster)



In [3]:
client

Client Scheduler: tcp://10.148.0.134:55958 Dashboard: http://10.148.0.134:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
# same but over all variables and faces
V = ["SSU","SSV"]
#V = V+["KPPhbl", "qsoceTAUX", "oceTAUY"]

In [5]:
def compute_store(v, ds, diag, op):
    #i, j = ut.get_ij_dims(ds)
    diag_zarr = diag+"_"+v
    if not ut.is_diagnostic(diag_zarr):
        out, tmp_zarr = ut.custom_distribute(ds, op, 
                                             tmp_dir=ut.scratch+"tmp/", 
                                             face=1, ) # **{j: 1080}
        ut.store_diagnostic(diag_zarr, out, overwrite=True)
        print("{} {} stored".format(v, diag))
    else:
        print("{} {} already exists".format(v, diag))

In [15]:
diags = {"prime_mean_sq": lambda x: ((x-x.mean("time"))**2).mean("time")
        #, "mean_square": lambda x: (x**2).mean("time"),
        }
        
for v in V:
    ds = ut.load_data(v)
    for diag, op in diags.items():
        compute_store(v, ds, diag, op)

data stored in /home/datawork-lops-osi/equinox/mit4320/diags/prime_mean_sq_SSU.zarr
SSU prime_mean_sq stored
data stored in /home/datawork-lops-osi/equinox/mit4320/diags/prime_mean_sq_SSV.zarr
SSV prime_mean_sq stored


In [6]:
ds_tot = ut.load_data(V)

In [7]:
grid = xr.open_zarr('/home1/datawork/xyu/grid/mit_grid.zarr')
ds_tot = xr.merge([ds_tot,grid])
#ds_tot['SSU'] = ds_tot['SSU'].rename({'i_g':'i'})
#ds_tot['SSV'] = ds_tot['SSV'].rename({'j_g':'j'})
# Rotation velocity field
ds_tot['U'] =  ds_tot.SSU.rename({'i_g': 'i'})*ds_tot.CS - ds_tot.SSV.rename({'j_g': 'j'})*ds_tot.SN
ds_tot['V'] =  ds_tot.SSU.rename({'i_g': 'i'})*ds_tot.SN + ds_tot.SSV.rename({'j_g': 'j'})*ds_tot.CS

ds_tot['lon'] = 0*ds_tot.face+ds_tot.XC
ds_tot['lat'] = 0*ds_tot.face+ds_tot.YC

#V['lon'] = 0*ds_tot.face+ds_tot.XC
#V['lat'] = 0*ds_tot.face+ds_tot.YC

In [10]:
ds_tot = ds_tot.drop('i_g')

In [9]:
ds_tot

<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, i_g: 4320, j: 4320, j_g: 4320, time: 8785)
Coordinates: (12/22)
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    niter    (time) int64 dask.array<chunksize=(200,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2011-11-15 2011-11-15T01:00:00 ... 2012-11-15
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    ...       ...
    dyC      (face, j_g, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    dyG      (face, j, i_g) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    rA       (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    rAs      (face, j_g, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    rAw      (face, j, i_g) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    rAz      (face, j_g, i_g) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
Data variables:
    SSU      (time, face, j, i_g) float32 dask.array<chunksize=(1, 1, 4320, 4320), meta=np.ndarray>
    SSV      (time, face, j_g, i) float32 dask.array<chunksize=(1, 1, 4320, 4320), meta=np.ndarray>
    U        (time, face, j, i) float32 dask.array<chunksize=(1, 1, 4320, 4320), meta=np.ndarray>
    V        (time, face, j, i) float32 dask.array<chunksize=(1, 1, 4320, 4320), meta=np.ndarray>
    lon      (face, j, i) float64 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    lat      (face, j, i) float64 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>

In [13]:
ds_tot = ds_tot.persist()

In [11]:
ds_tot['velocity_amplitude'] = np.sqrt(ds_tot['SSU']**2+ds_tot['SSV']**2)

In [8]:
ds_tot = ds_tot.drop(['SSU','SSV'])

In [9]:
ds_tot

<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, i_g: 4320, j: 4320, j_g: 4320, time: 8785)
Coordinates: (12/22)
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    niter    (time) int64 dask.array<chunksize=(200,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2011-11-15 2011-11-15T01:00:00 ... 2012-11-15
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    ...       ...
    dyC      (face, j_g, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    dyG      (face, j, i_g) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    rA       (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    rAs      (face, j_g, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    rAw      (face, j, i_g) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    rAz      (face, j_g, i_g) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
Data variables:
    U        (time, face, j, i) float32 dask.array<chunksize=(1, 1, 4320, 4320), meta=np.ndarray>
    V        (time, face, j, i) float32 dask.array<chunksize=(1, 1, 4320, 4320), meta=np.ndarray>
    lon      (face, j, i) float64 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    lat      (face, j, i) float64 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>

In [ ]:
diags = {"prime_mean_sq": lambda x: ((x-x.mean("time"))**2).mean("time")
        #, "mean_square": lambda x: (x**2).mean("time"),
        }
V = ['U','V']        
for v in V:
    for diag, op in diags.items():
        compute_store(v, ds_tot, diag, op)

In [9]:
_ds = xr.Dataset({'SSU':U,'SSV':V})

In [10]:
_ds['velocity_amplitude'] = np.sqrt(_ds['SSU']**2+_ds['SSV']**2)

In [11]:
_ds

<xarray.Dataset>
Dimensions:             (face: 13, i: 4320, j: 4320, time: 8785)
Coordinates: (12/19)
  * face                (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i                   (i) int64 0 1 2 3 4 5 ... 4314 4315 4316 4317 4318 4319
  * j                   (j) int64 0 1 2 3 4 5 ... 4314 4315 4316 4317 4318 4319
    niter               (time) int64 dask.array<chunksize=(200,), meta=np.ndarray>
  * time                (time) datetime64[ns] 2011-11-15 ... 2012-11-15
    dxC                 (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    ...                  ...
    rA                  (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    dxG                 (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    dyC                 (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    rAs                 (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    lon                 (face, j, i) float64 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    lat                 (face, j, i) float64 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
Data variables:
    SSU                 (time, face, j, i) float32 dask.array<chunksize=(1, 1, 4320, 4320), meta=np.ndarray>
    SSV                 (time, face, j, i) float32 dask.array<chunksize=(1, 1, 4320, 4320), meta=np.ndarray>
    velocity_amplitude  (time, face, j, i) float32 dask.array<chunksize=(1, 1, 4320, 4320), meta=np.ndarray>

In [12]:
_ds = _ds.persist()

In [ ]:
_ds_sub = _ds.sel(face=0)

In [ ]:
out, tmp_zarr = ut.custom_distribute(_ds, op, tmp_dir=ut.scratch+"tmp/", face=1, )

In [19]:
V = ["SSU", "SSV"]
D = ["mean"]
ds = xr.merge([ut.load_diagnostic(d+"_"+v).rename({v:d+"_"+v}) 
               for v in V for d in D])

grid = xr.open_zarr('/home1/datawork/xyu/grid/mit_grid.zarr')
ds = xr.merge([ds,grid])
ds = ds#.isel(i = slice(0,None,4), j = slice(0,None,4), i_g = slice(0,None,4), j_g = slice(0,None,4))
ds['lon'] = 0*ds.face+ds.XC
ds['lat'] = 0*ds.face+ds.YC
ds = ds.compute()

distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/core.py", line 497, in handle_comm
    result = handler(comm, **msg)
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/scheduler.py", line 3820, in heartbeat_worker
    ws._executing = {
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/scheduler.py", line 3821, in <dictcomp>
    parent._tasks[key]: duration for key, duration in executing.items()
KeyError: "('xarray-XC-4d5edf5dc3a3b8d6b4efeb7c15e9610c', 3, 0, 0)"
distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/core.py", line 497, in handle_comm
    result = handler(comm, **msg)
  File "/home1/datahome/

KeyboardInterrupt: 

In [ ]:
# Rotation velocity field
U_mean =  ds.mean_SSU.rename({'i_g': 'i'})*ds.CS - ds.mean_SSV.rename({'j_g': 'j'})*ds.SN
V_mean =  ds.mean_SSU.rename({'i_g': 'i'})*ds.SN + ds.mean_SSV.rename({'j_g': 'j'})*ds.CS

U_mean['lon'] = 0*ds.face+ds.XC
U_mean['lat'] = 0*ds.face+ds.YC

V_mean['lon'] = 0*ds.face+ds.XC
V_mean['lat'] = 0*ds.face+ds.YC

In [ ]:
# Mean Eulerian field Dataset
ds_mean = xr.Dataset({'meridional_velocity_mean':V_mean,'zonal_velocity_mean':U_mean})

In [19]:
_ds['velocity_amplitude'] = np.sqrt(_ds['SSU']**2+_ds['SSV']**2)

In [20]:
_ds

<xarray.Dataset>
Dimensions:             (face: 13, i: 4320, j: 4320, time: 8785)
Coordinates: (12/19)
  * face                (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i                   (i) int64 0 1 2 3 4 5 ... 4314 4315 4316 4317 4318 4319
  * j                   (j) int64 0 1 2 3 4 5 ... 4314 4315 4316 4317 4318 4319
    niter               (time) int64 dask.array<chunksize=(200,), meta=np.ndarray>
  * time                (time) datetime64[ns] 2011-11-15 ... 2012-11-15
    dxC                 (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    ...                  ...
    rA                  (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    dxG                 (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    dyC                 (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    rAs                 (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    lon                 (face, j, i) float64 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    lat                 (face, j, i) float64 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
Data variables:
    SSU                 (time, face, j, i) float32 dask.array<chunksize=(1, 1, 4320, 4320), meta=np.ndarray>
    SSV                 (time, face, j, i) float32 dask.array<chunksize=(1, 1, 4320, 4320), meta=np.ndarray>
    velocity_amplitude  (time, face, j, i) float32 dask.array<chunksize=(1, 1, 4320, 4320), meta=np.ndarray>

In [21]:
diags = {"mean": lambda x: x.mean("time")
#         ,"mean_square": lambda x: (x**2).mean("time"),
        }
V = ['velocity_amplitude']        
for v in V:
    for diag, op in diags.items():
        compute_store(v, _ds, diag, op)

KeyboardInterrupt: 

distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/core.py", line 497, in handle_comm
    result = handler(comm, **msg)
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/scheduler.py", line 3820, in heartbeat_worker
    ws._executing = {
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/scheduler.py", line 3821, in <dictcomp>
    parent._tasks[key]: duration for key, duration in executing.items()
KeyError: "('xarray-SSU-dc0963372fb5a236f343d86b2668b032', 5992, 0, 0, 0)"
distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/core.py", line 497, in handle_comm
    result = handler(comm, **msg)
  File "/home1/da

In [ ]:
V = ["Eta", "SSU", "SSV", "SST", "SSS"]
D = ["mean", "mean_square"]
ds = xr.merge([ut.load_diagnostic(d+"_"+v).rename({v:d+"_"+v}) 
               for v in V for d in D])

grid = xr.open_zarr('/home1/datawork/xyu/grid/mit_grid.zarr')
ds = xr.merge([ds,grid])
ds = ds.isel(i = slice(0,None,4), j = slice(0,None,4), i_g = slice(0,None,4), j_g = slice(0,None,4))
ds['lon'] = 0*ds.face+ds.XC
ds['lat'] = 0*ds.face+ds.YC
ds.load()

In [11]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
